# Infer RFHLTH for unseen data
In this notebook we demonstrate inference on unseen data. In this case we use synthetic data but could be new data for real individuals. 

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PopHealthRisk").getOrCreate()

In [0]:
from pyspark.ml.classification import GBTClassificationModel

loaded_model = GBTClassificationModel.load("/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/models/gbt-1")

In [0]:
# Note that loading via MLflow does not work on serverless compute, but if set up correctly, we could load like this 
#import mlflow.spark

#loaded_model = mlflow.spark.load_model("models:/pophealthrisk.pophealthrisk.phr_rf@prod")

In [0]:
# Load synthetic data and assemble for inference
from pyspark.ml.feature import VectorAssembler

df = spark.read.parquet('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/synthetic_data.parquet', header=True, inferSchema=True)
feature_cols=["_AGEG5YR_clean","EDUCA_clean",'_BMI5', '_SMOKER3_clean', 'DRNKANY6_clean','INCOME3_clean','num_conditions','income_adj_pov']

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

df_assembled = assembler.transform(df)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
# Infer
predictions = loaded_model.transform(df_assembled)

In [0]:
# write prediction labels
predictions.select('prediction').write.parquet('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/synthetic_data_inferred.parquet', mode="overwrite")